In [ ]:
import pyspark as ps
import polars as pl

from pyspark.sql import functions as F

In [ ]:
spark = ps.sql.SparkSession.builder.getOrCreate()

In [ ]:
kgml = pl.read_parquet('gs://mtrx-us-central1-hub-dev-storage/kedro/data/releases/v0.11.2/datasets/integration/int/kgml_xdtd_ground_truth/edges.norm/')

In [ ]:
off_label = pl.read_parquet('gs://mtrx-us-central1-hub-dev-storage/kedro/data/releases/v0.11.2/datasets/integration/int/off_label/edges.norm/')

In [ ]:
db_gt = pl.read_parquet('gs://mtrx-us-central1-hub-prod-storage/kedro/data/releases/v0.11.11-prod/datasets/integration/int/drugbank_ground_truth/edges.norm/')

In [ ]:
db_gt

In [ ]:
db_gt.filter(pl.col("y") == 1).select("subject", "object", "y")

In [ ]:
kgml_df = spark.createDataFrame(kgml.to_pandas())
kgml_df.show()

In [ ]:
db_gt = spark.createDataFrame(db_gt.to_pandas())

In [ ]:
from functools import reduce

In [ ]:
datasets_to_include = {
    'kgml_ground_truth': 
    {"positives": True, "negatives": False}, 
    'db_ground_truth': 
    {"positives": True, "negatives": True}
    }

In [ ]:
all_datasets = {
    "kgml_ground_truth": kgml_df,
    "db_ground_truth": db_gt
}

In [ ]:
included_dataset_list = []
for dataset_name, dataset in all_datasets.items():
    y_values_required = [
        y for y, pair_type in zip([0,1], ["negatives", "positives"]) if datasets_to_include[dataset_name][pair_type]
        ] 
    if len(y_values_required) > 0:
        included_dataset_list.append(
            dataset
            .filter(F.col("y").isin(y_values_required))
            .select(F.col("subject").alias("drug_id"), F.col("object").alias("disease_id"))
        )
reduce(lambda df1, df2: df1.union(df2), included_dataset_list).distinct().show()

In [ ]:
y_values_required = {
    name : [
        y for y, pair_type in zip([0,1], ["negatives", "positives"]) 
        if datasets_to_include[name][pair_type]
    ] 
    for name in all_datasets.keys()
}
dataframes_to_concatenate = [
        df
        .filter(F.col("y").isin(y_values_required[name]))
        .select(F.col("subject").alias("drug_id"), F.col("object").alias("disease_id"))
        for name, df in all_datasets.items()
        if len(y_values_required[name]) > 0
]
reduce(lambda df1, df2: df1.union(df2), dataframes_to_concatenate).distinct().show()

In [ ]:
y_values = {dataset_name : []
}


In [ ]:
for dataset

In [ ]:
datasets_to_include = [
    dataset
    .filter(F.col("y").isin(y_values[dataset_name]))
    .select(F.col("subject").alias("drug_id"), F.col("object").alias("disease_id"))
    for dataset_name, dataset in all_datasets.items()
    if len(y_values[dataset_name]) > 0
]

In [ ]:
reduce(lambda df1, df2: df1.union(df2), [kgml_df, db_gt])

In [ ]:
F.union()

In [ ]:
kgml_df